# Women’s World Cup 2019

## Requisiti
Neo4j, Python 3.11

In [ ]:
% pip install py2neo neo4j-driver

Connessione e autenticazione:

In [3]:
from py2neo import Graph

graph = Graph("bolt://52.54.222.44:7687", auth=("neo4j", "effort-watts-restriction"))
cypher = """
MATCH (t:Tournament {year: 2019})<-[:PARTICIPATED_IN]-(team)
RETURN team.name as team
"""
print (graph. run(cypher))

 team      
-----------
 Argentina 
 Australia 
 Brazil    



In [12]:
print(dir(graph))

['__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_procedures', '_update', 'auto', 'begin', 'call', 'commit', 'create', 'delete', 'delete_all', 'evaluate', 'exists', 'match', 'match_one', 'merge', 'name', 'nodes', 'pull', 'push', 'query', 'read', 'relationships', 'rollback', 'run', 'schema', 'separate', 'service', 'update']
